Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/tabular-data/explain-binary-classification-local.png)

# Explain binary classification model predictions
_**This notebook showcases how to use the Azure Machine Learning Interpretability SDK to explain and visualize a binary classification model predictions.**_


## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Run model explainer locally at training time](#Explain)
    1. Train a binary classification model
    1. Explain the model
        1. Generate global explanations
        1. Generate local explanations
1. [Visualize results](#Visualize)
1. [Next steps](#Next)

## Introduction

This notebook illustrates how to explain a binary classification model predictions locally at training time without contacting any Azure services.
It demonstrates the API calls that you need to make to get the global and local explanations and a visualization dashboard that provides an interactive way of discovering patterns in data and explanations.

We will showcase three tabular data explainers: TabularExplainer (SHAP), MimicExplainer (global surrogate), and PFIExplainer.

| ![Interpretability Toolkit Architecture](./img/interpretability-architecture.png) |
|:--:|
| *Interpretability Toolkit Architecture* |

Problem: Breast cancer diagnosis classification with scikit-learn (run model explainer locally)

1. Train a SVM classification model using Scikit-learn
2. Run 'explain_model' globally and locally with full dataset in local mode, which doesn't contact any Azure services.
3. Visualize the global and local explanations with the visualization dashboard.
---

Setup: If you are using Jupyter notebooks, the extensions should be installed automatically with the package.
If you are using Jupyter Labs run the following command:
```
(myenv) $ jupyter labextension install @jupyter-widgets/jupyterlab-manager
```


## Explain

### Run model explainer locally at training time

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn import svm

# Explainers:
# 1. SHAP Tabular Explainer
#from interpret.ext.blackbox import TabularExplainer
from interpret.ext.blackbox import TabularExplainer

# OR

# 2. Mimic Explainer
#from interpret.ext.blackbox import MimicExplainer

# You can use one of the following four interpretable models as a global surrogate to the black box model
#from interpret.ext.glassbox import LGBMExplainableModel
#from interpret.ext.glassbox import LinearExplainableModel
#from interpret.ext.glassbox import SGDExplainableModel
#from interpret.ext.glassbox import DecisionTreeExplainableModel

# OR

# 3. PFI Explainer
#from interpret.ext.blackbox import PFIExplainer 

### Load the breast cancer diagnosis data

In [2]:
breast_cancer_data = load_breast_cancer()
classes = breast_cancer_data.target_names.tolist()

In [3]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(breast_cancer_data.data, breast_cancer_data.target, test_size=0.2, random_state=0)

### Train a SVM classification model, which you want to explain

In [4]:
clf = svm.SVC(gamma=0.001, C=100., probability=True)
model = clf.fit(x_train, y_train)

### Explain predictions on your local machine

In [5]:
# 1. Using SHAP TabularExplainer
explainer = TabularExplainer(model, 
                             x_train, 
                             features=breast_cancer_data.feature_names, 
                             classes=classes)




# 2. Using MimicExplainer
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns. 
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
# explainer = MimicExplainer(model, 
#                            x_train, 
#                            LGBMExplainableModel, 
#                            augment_data=True, 
#                            max_num_of_augmentations=10, 
#                            features=breast_cancer_data.feature_names, 
#                            classes=classes)





# 3. Using PFIExplainer

# Use the parameter "metric" to pass a metric name or function to evaluate the permutation. 
# Note that if a metric function is provided a higher value must be better.
# Otherwise, take the negative of the function or set the parameter "is_error_metric" to True.
# Default metrics: 
# F1 Score for binary classification, F1 Score with micro average for multiclass classification and
# Mean absolute error for regression

# explainer = PFIExplainer(model, 
#                          features=breast_cancer_data.feature_names, 
#                          classes=classes)

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\linear.py:49: UserWarning: The default value for feature_dependence has been changed to "independent"!
  warnings.warn('The default value for feature_dependence has been changed to "independent"!')


### Generate global explanations
Explain overall model predictions (global explanation)

In [7]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(x_test)

# Note: if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_test, true_labels=y_test)

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg

In [8]:
# Sorted SHAP values
print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
# Corresponding feature names
print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
# Feature ranks (based on original order of features)
print('global importance rank: {}'.format(global_explanation.global_importance_rank))

# Note: PFIExplainer does not support per class explanations
# Per class feature names
print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))
# Per class feature importance values
print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

ranked global importance values: [0.10620339859352548, 0.09057725548032011, 0.07858883129813168, 0.06455387096467935, 0.04047534875941913, 0.01196496211139434, 0.0018874689913705718, 0.0010996141064060208, 0.0010426182878993588, 0.0008849819719460648, 0.0008743574634560063, 0.0008570889665395913, 0.0008545370936375175, 0.0008474046699653872, 0.0007987760580950221, 0.0007855200482798693, 0.0007788162133749007, 0.0007324278628529206, 0.0007152477607690403, 0.0006927033906102217, 0.0006875814046230236, 0.0006667281177449228, 0.000653332321054579, 0.0005838490452496105, 0.0005834733159074182, 0.0005813771724628884, 0.000533089248111674, 0.0005210968112428864, 0.0004592158057940878, 0.00027403667275721025]
ranked global importance names: ['worst area', 'worst perimeter', 'mean area', 'mean perimeter', 'area error', 'worst texture', 'mean texture', 'worst radius', 'mean radius', 'mean concave points', 'mean concavity', 'symmetry error', 'smoothness error', 'mean symmetry', 'compactness error

In [9]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

global importance rank: {'worst area': 0.10620339859352548, 'worst perimeter': 0.09057725548032011, 'mean area': 0.07858883129813168, 'mean perimeter': 0.06455387096467935, 'area error': 0.04047534875941913, 'worst texture': 0.01196496211139434, 'mean texture': 0.0018874689913705718, 'worst radius': 0.0010996141064060208, 'mean radius': 0.0010426182878993588, 'mean concave points': 0.0008849819719460648, 'mean concavity': 0.0008743574634560063, 'symmetry error': 0.0008570889665395913, 'smoothness error': 0.0008545370936375175, 'mean symmetry': 0.0008474046699653872, 'compactness error': 0.0007987760580950221, 'worst concavity': 0.0007855200482798693, 'worst smoothness': 0.0007788162133749007, 'worst symmetry': 0.0007324278628529206, 'concave points error': 0.0007152477607690403, 'radius error': 0.0006927033906102217, 'mean fractal dimension': 0.0006875814046230236, 'mean compactness': 0.0006667281177449228, 'worst concave points': 0.000653332321054579, 'worst compactness': 0.0005838490

### Explain overall model predictions as a collection of local (instance-level) explanations

In [10]:
# feature shap values for all features and all data points in the training data
print('local importance values: {}'.format(global_explanation.local_importance_values))

local importance values: [[[0.0, 0.0, -0.011511761572632229, 0.06538674773095213, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06090832197403681, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0085890545825857, 0.03133202892333736, 0.1089445689242095, -0.09153386058382677, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.040385492206170104, 0.0, 0.0, 0.0, -0.005592557469332977, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.018315436349629385, 0.0, -0.009736603253360718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04468417727899496, -0.07608840232729655, -0.22121681077243238, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.023263164277546017, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.027764180895707008, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.022727888391607665, -0.08037020699225794, -0.17897880512330933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.042586057433512986, 0.16083458376963056, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0250962679730376, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.049647224928413836, 0.24994007

### Generate local explanations
Explain local data points (individual instances)

In [11]:
# Note: PFIExplainer does not support local explanations
# You can pass a specific data point or a group of data points to the explain_local function

# E.g., Explain the first data point in the test set
instance_num = 0
local_explanation = explainer.explain_local(x_test[instance_num,:])

C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\bezeran\AppData\Local\conda\conda\envs\interp4\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


In [12]:
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = clf.predict(x_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]

print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

local importance values: [0.09087460343871251, 0.06182973483410147, 0.014925285456356782, 0.00714261012663342, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.03447585966423778, -0.06906908895113692, -0.1043476321058468]
local importance names: ['worst area', 'area error', 'mean perimeter', 'mean radius', 'worst symmetry', 'mean texture', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'worst fractal dimension', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'smoothness error', 'worst texture', 'mean area', 'worst perimeter']


## Visualize
Load the visualization dashboard

In [13]:
from interpret_community.widget import ExplanationDashboard

In [15]:
ExplanationDashboard(global_explanation, model, datasetX=x_test)

ValueError: Unsupported local explanation type

In [19]:
global_explanation.data(key=-1)

{'overall': {'names': ['mean radius',
   'mean texture',
   'mean perimeter',
   'mean area',
   'mean smoothness',
   'mean compactness',
   'mean concavity',
   'mean concave points',
   'mean symmetry',
   'mean fractal dimension',
   'radius error',
   'texture error',
   'perimeter error',
   'area error',
   'smoothness error',
   'compactness error',
   'concavity error',
   'concave points error',
   'symmetry error',
   'fractal dimension error',
   'worst radius',
   'worst texture',
   'worst perimeter',
   'worst area',
   'worst smoothness',
   'worst compactness',
   'worst concavity',
   'worst concave points',
   'worst symmetry',
   'worst fractal dimension'],
  'scores': [0.0010426182878993588,
   0.0018874689913705718,
   0.06455387096467935,
   0.07858883129813168,
   0.00027403667275721025,
   0.0006667281177449228,
   0.0008743574634560063,
   0.0008849819719460648,
   0.0008474046699653872,
   0.0006875814046230236,
   0.0006927033906102217,
   0.0005210968112428

In [16]:
help(ExplanationDashboard)

Help on class ExplanationDashboard in module interpret_community.widget.ExplanationDashboard:

class ExplanationDashboard(builtins.object)
 |  The dashboard class, wraps the dashboard component.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, explanationObject, model=None, *, datasetX=None, trueY=None, classes=None, features=None)
 |      Initialize the Explanation Dashboard.
 |      
 |      :param explanationObject: An object that represents an explanation.
 |      :type explanationObject: ExplanationMixin
 |      :param model: An object that represents a model. It is assumed that for the classification case
 |          it has a method of predict_proba() returning the prediction probabilities for each
 |          class and for the regression case a method of predict() returning the prediction value.
 |      :type model: object
 |      :param datasetX:  A matrix of feature vector examples (# examples x # features), the same samples
 |          used to build the explanationObjec

## Next
Learn about other use cases of the explain package on a:
       
1. [Training time: regression problem](./explain-regression-local.ipynb)
1. [Training time: multiclass classification problem](./explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](./simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](./advanced-feature-transformations-explain-local.ipynb)
1. [Save model explanations via Azure Machine Learning Run History](../azure-integration/run-history/save-retrieve-explanations-run-history.ipynb)
1. [Run explainers remotely on Azure Machine Learning Compute (AMLCompute)](../azure-integration/remote-explanation/explain-model-on-amlcompute.ipynb)
1. Inferencing time: deploy a classification model and explainer:
    1. [Deploy a locally-trained model and explainer](../azure-integration/scoring-time/train-explain-model-locally-and-deploy.ipynb)
    1. [Deploy a remotely-trained model and explainer](../azure-integration/scoring-time/train-explain-model-on-amlcompute-and-deploy.ipynb)
